In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import random
from scipy.fft import fft, ifft

In [2]:
import sys
sys.path.append('C:/Users/DimiP/Documents/GitHub/ULG/COURS/basic-MPC/src')
from features import process_utils
%reload_ext autoreload

In [3]:
import importlib
importlib.reload(process_utils)

<module 'features.process_utils' from 'C:/Users/DimiP/Documents/GitHub/ULG/COURS/basic-MPC/src\\features\\process_utils.py'>

In [4]:
#création des dicts de dataframe
path='c:\\Users\\DimiP\\Documents\\GitHub\\ULG\\COURS\\basic-MPC\\data\\raw'
csv_files = glob.glob(os.path.join(path, "*.csv"))
data={}
for file in csv_files:
    
    p=Path(file)
    filename=p.with_suffix('').stem
    finalfilename=filename.replace("temperature_", "")
    data[finalfilename]=pd.read_csv(p)
    data[finalfilename].index=data[finalfilename]['time'].apply(lambda x: process_utils.convertdate(x))


    

In [6]:
data.keys()

dict_keys(['big_df', 'pv_production_load', 'bathroom', 'bedroom_1', 'bedroom_2', 'bedroom_3', 'diningroom', 'heating_system', 'kitchen', 'livingroom', 'outside'])

In [7]:
data['heating_system'].head(2)

,time,water_pressure,water_temperature
time,,,
2020-05-24 17:20:03,2020-05-24T17:20:03Z,1.3,24
2020-05-24 17:25:03,2020-05-24T17:25:03Z,1.6,24


In [8]:
data['pv_production_load'].columns = ['time', 'L1_PV', 'L2_PV', 'L3_PV', 'L1_Load', 'L2_Load','L3_Load']

In [9]:
pv=data['pv_production_load'].to_numpy()
heating=data['heating_system'].to_numpy()

In [10]:
#dataframe avec index seul

big_df = pd.DataFrame(index=data['kitchen'].index)
#big_df['total sun'] = np.abs(pv[:,1])+np.abs(pv[:,2])+np.abs(pv[:,3])
#big_df['pv load'] = np.abs(pv[:,4])+np.abs(pv[:,5])+np.abs(pv[:,6])

In [203]:
#data['pv_production_load']['datetime_obj']=data['pv_production_load'].apply(lambda x: process_utils.infodate(x['time'])[0],axis=1)


In [11]:
date_array = [process_utils.datetimeobj(x) for x in big_df.index]


In [12]:
df_add = pd.DataFrame(data=date_array,columns=['perc_D','week_D','number_D','year'],index=big_df.index)

In [13]:
big_df = pd.concat([big_df,df_add], axis=1)

In [14]:
mesT_df=pd.concat([data[key]['current_value'].dropna() for key in ['bathroom', 'bedroom_1', 'bedroom_2', 'bedroom_3',
 'diningroom', 'kitchen', 'livingroom']],axis=1)
setT_df=pd.concat([data[key]['setpoint'] for key in ['bathroom', 'bedroom_1', 'bedroom_2', 'bedroom_3',
 'diningroom', 'kitchen', 'livingroom']],axis=1)

In [15]:
big_df['multizone_mesT']=mesT_df.apply(list,axis=1)
big_df['multizone_setT']=setT_df.apply(list,axis=1)
big_df['outside_mesT']=data['outside']['current_value']
finaldf=big_df[big_df.index>pd.to_datetime('2020-05-24 19:40:03')]


In [18]:
def sum_nonneg(x):
    return sum(max(i,0) for i in x)
#data['pv_production_load']['PV'] = data['pv_production_load'][['L1_PV','L2_PV','L3_PV']].apply(lambda x: sum_nonneg(list(x)))
#data['pv_production_load']['PV_load'] = data['pv_production_load'][['L1_Load','L2_Load','L3_Load']].apply(lambda x: sum_nonneg(list(x)))


In [19]:
pv_df=data['pv_production_load'].reindex(index=big_df.index,method='nearest')
heating_df=data['heating_system'].reindex(index=big_df.index,method='nearest')
#[data['pv_production_load'].index > big_df.index.min() ]

In [20]:
pv_df.head(2)

,time,L1_PV,L2_PV,L3_PV,L1_Load,L2_Load,L3_Load
time,,,,,,,
2020-05-24 17:15:03,2020-05-22T08:24:02Z,240.0,124.0,241.0,22.0,229.0,91
2020-05-24 17:20:03,2020-05-22T08:24:02Z,240.0,124.0,241.0,22.0,229.0,91


In [21]:
heating_df.head(2)

,time,water_pressure,water_temperature
time,,,
2020-05-24 17:15:03,2020-05-24T17:20:03Z,1.3,24
2020-05-24 17:20:03,2020-05-24T17:20:03Z,1.3,24


In [23]:
pv_df.drop_duplicates(subset='time')
heating_df.drop_duplicates(subset='time')
big_df['PV'] = pv_df[['L1_PV','L2_PV','L3_PV']].apply(lambda x: sum_nonneg(x),axis=1)
big_df['PV_load'] = pv_df[['L1_Load','L2_Load','L3_Load']].apply(lambda x: sum_nonneg(x),axis=1)
big_df['waterP']=heating_df['water_pressure']
big_df['waterT']=heating_df['water_temperature']

In [24]:
def averageT(Tlist):
    return sum(Tlist) / len(Tlist)
big_df['singlezone_mesT'] = big_df['multizone_mesT'].apply(lambda x : [averageT(x)])
big_df['singlezone_setT'] = big_df['multizone_setT'].apply(lambda x : [averageT(x)])

In [25]:
lst=big_df.index
big_df['singlezone_nextT'] = pd.Series()
big_df['multizone_nextT'] = pd.Series()
big_df['singlezone_nextT']=big_df['singlezone_mesT'].shift(-1)
big_df['multizone_nextT'] = big_df['multizone_mesT'].shift(-1)


emptydf=pd.DataFrame(index=big_df.index)
emptydf['timestamp']=big_df.index
big_df['timestep']=emptydf['timestamp'].diff().dt.total_seconds().shift(-1)


C:\Users\DimiP\.conda\envs\pacman\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\DimiP\.conda\envs\pacman\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
big_df=big_df.dropna()
big_df.head(5)

,perc_D,week_D,number_D,year,multizone_mesT,multizone_setT,outside_mesT,PV,PV_load,waterP,waterT,singlezone_mesT,singlezone_setT,singlezone_nextT,multizone_nextT,timestep
time,,,,,,,,,,,,,,,,
2020-05-24 19:40:03,0.819479,sunday,145,2020,"[21.8, 22.5, 22.4, 21.7, 22.0, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,605.0,342.0,1.3,25,[22.185714285714283],[15.857142857142858],[22.185714285714283],"[21.8, 22.5, 22.4, 21.7, 22.0, 22.0, 22.9]",300.0
2020-05-24 19:45:03,0.822951,sunday,145,2020,"[21.8, 22.5, 22.4, 21.7, 22.0, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,605.0,342.0,1.3,25,[22.185714285714283],[15.857142857142858],[22.12857142857143],"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]",300.0
2020-05-24 19:50:03,0.826424,sunday,145,2020,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,605.0,342.0,1.3,25,[22.12857142857143],[15.857142857142858],[22.12857142857143],"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]",300.0
2020-05-24 19:55:03,0.829896,sunday,145,2020,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,605.0,342.0,1.3,25,[22.12857142857143],[15.857142857142858],[22.12857142857143],"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]",301.0
2020-05-24 20:00:04,0.833380,sunday,145,2020,"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]","[16.0, 16.0, 16.0, 15.0, 16.0, 16.0, 16.0]",14.0,605.0,342.0,1.3,25,[22.12857142857143],[15.857142857142858],[22.12857142857143],"[21.8, 22.4, 22.4, 21.7, 21.7, 22.0, 22.9]",300.0


In [27]:
big_df.to_csv('C:/Users/DimiP/Documents/GitHub/ULG/COURS/basic-MPC/data/raw/big_df.csv')